# 0. 필요 모듈 불러오기

In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd

rs = 123
np.random.seed(rs)
random.seed(rs)

# DataFrame 출력 최대 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#머신러닝
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.linear_model import *

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# 데이터 정리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV


from sklearn.ensemble import *
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xg

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# 1. 데이터 로드

In [2]:
dataset_gid = gpd.read_file('model_dataset.geojson', driver= 'GeoJSON')
dataset_gid

,gid,acci_cnt,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry
0,다바866110,0,0.0,0.00,0.0,0,0,0,0,0,0,0,0,13,0,146.59,4,8283.000,2.74000,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,다바823157,0,0.0,0.00,0.0,3,4,2,3,0,0,14,0,267,1,5944.98,0,1181.625,38.47125,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,다바928260,0,0.0,0.00,0.0,6,5,0,4,3,0,14,0,78,2,1487.49,40,3783.750,70.80250,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,다바931203,2,2.1,1.65,0.0,1,3,1,1,2,0,5,0,0,0,1291.19,409,11596.000,98.50000,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,다바836109,0,0.0,0.00,0.0,2,0,0,2,0,0,6,0,396,0,12875.52,0,678.875,65.44500,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,다바911108,1,0.0,0.00,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,286.500,94.86000,"MULTIPOLYGON (((127.40088 36.29447, 127.40088 ..."
12175,다바968288,0,0.0,0.00,0.0,0,0,0,0,0,0,2,0,0,0,0.00,0,2515.250,3.82500,"MULTIPOLYGON (((127.46429 36.45678, 127.46429 ..."
12176,다바906266,0,0.0,0.00,0.0,0,0,0,0,0,0,3,0,0,0,0.00,0,16135.250,5.13750,"MULTIPOLYGON (((127.39512 36.43691, 127.39512 ..."
12177,다바906193,0,0.0,0.00,0.0,0,0,0,0,0,0,2,1,0,0,0.00,0,14414.500,48.29500,"MULTIPOLYGON (((127.39521 36.37110, 127.39521 ..."


## gid 엔코딩

In [3]:
dataset_encode = pd.DataFrame()
for col in tqdm(dataset_gid.columns):
    if type(dataset_gid[col][0]) == str:
        if col == 'gid':
            gid_lbe = LabelEncoder()
            gid_lbe.fit(dataset_gid[col])
            dataset_encode[col] = gid_lbe.transform(dataset_gid[col])
        else : 
            lbe = LabelEncoder()
            lbe.fit(dataset_gid[col])
            dataset_encode[col] = lbe.transform(dataset_gid[col])
    else:
        dataset_encode[col] = dataset_gid[col]
dataset_encode

100%|██████████| 20/20 [00:00<00:00, 700.01it/s]


,gid,acci_cnt,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry
0,3173,0,0.0,0.00,0.0,0,0,0,0,0,0,0,0,13,0,146.59,4,8283.000,2.74000,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,507,0,0.0,0.00,0.0,3,4,2,3,0,0,14,0,267,1,5944.98,0,1181.625,38.47125,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,8410,0,0.0,0.00,0.0,6,5,0,4,3,0,14,0,78,2,1487.49,40,3783.750,70.80250,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,8682,2,2.1,1.65,0.0,1,3,1,1,2,0,5,0,0,0,1291.19,409,11596.000,98.50000,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,1192,0,0.0,0.00,0.0,2,0,0,2,0,0,6,0,396,0,12875.52,0,678.875,65.44500,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,6668,1,0.0,0.00,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,286.500,94.86000,"MULTIPOLYGON (((127.40088 36.29447, 127.40088 ..."
12175,11088,0,0.0,0.00,0.0,0,0,0,0,0,0,2,0,0,0,0.00,0,2515.250,3.82500,"MULTIPOLYGON (((127.46429 36.45678, 127.46429 ..."
12176,6254,0,0.0,0.00,0.0,0,0,0,0,0,0,3,0,0,0,0.00,0,16135.250,5.13750,"MULTIPOLYGON (((127.39512 36.43691, 127.39512 ..."
12177,6222,0,0.0,0.00,0.0,0,0,0,0,0,0,2,1,0,0,0.00,0,14414.500,48.29500,"MULTIPOLYGON (((127.39521 36.37110, 127.39521 ..."


In [4]:
# 결과 저장
dataset_encode.to_csv('dataset_encode.csv', header = True, index = False)

## train / test split

In [5]:
X = dataset_encode.drop(['acci_cnt','geometry'], axis=1)
y = dataset_encode['acci_cnt']

# 7:3 비율로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 1234)

# 3. Modeling

## 3-1. Linear Regression

In [6]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_train)

print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = lm.predict(X_test)
acc_linreg = metrics.r2_score(y_test, y_test_pred)
print('\nTest score')
print('R^2:', acc_linreg)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.40425162756596733
Adjusted R^2: 0.4029909326795562
MAE: 1.7500580815890645
MSE: 11.89723068638367
RMSE: 3.4492362468209787

Test score
R^2: 0.35983551946105197
Adjusted R^2: 0.35666551653128553
MAE: 1.702608723901062
MSE: 9.256175783081611
RMSE: 3.042396388224521


# Ensanbles

## 3-2. Random Forest

In [7]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_train)

print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = rf.predict(X_test)

acc_rf = metrics.r2_score(y_test, y_test_pred)
print('\nTest score')
print('R^2:', acc_rf)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.9577373195291147
Adjusted R^2: 0.9576478852182193
MAE: 0.3880023460410557
MSE: 0.8439953548387096
RMSE: 0.9186921980939587

Test score
R^2: 0.7099557535787602
Adjusted R^2: 0.7085194959623689
MAE: 0.9420799124247399
MSE: 4.193766776135742
RMSE: 2.047868837629925


Feature importance 확인

In [8]:
f1 = pd.DataFrame({'data': X.columns, 'importance': rf.feature_importances_}).sort_values(by='importance', ascending=False)
f1

,data,importance
1,일강수량(mm),0.346736
3,안개 계속시간(hr),0.103286
16,전체_추정교통량,0.091713
2,일 최심적설(cm),0.079166
5,차량신호등,0.077826
10,교통안전표지,0.050116
4,보행신호등,0.047856
17,혼잡시간강도,0.038716
14,건물면적,0.032287
0,gid,0.023140


## 3-3. XGBRegressor

In [9]:
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear'] #default gbtree 
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
base_score = [0.1,0.25,0.5]
n_estimators = [5, 10, 15, 20, 25]


hyperparameter_grid = {'n_estimators': n_estimators,
                       'max_depth':max_depth,
                       'learning_rate':learning_rate,
                       'min_child_weight':min_child_weight,
                       'booster':booster,
                       'base_score':base_score}

regressor = XGBRegressor()
gd=RandomizedSearchCV(estimator=regressor,param_distributions=hyperparameter_grid,verbose=True)

gd.fit(X_train,y_train)

print(gd.best_score_)
print(gd.best_estimator_)
best_xgb = gd.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[02:54:12] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:54:12] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:54:13] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[02:54:13] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:54:13] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:54:13] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   14.2s finished


0.6863802836091523
XGBRegressor(base_score=0.1, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=15, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)


In [10]:
best_xgb.fit(X_train,y_train)

y_pred = best_xgb.predict(X_train)
print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = best_xgb.predict(X_test)
print('\nTest score')
acc_xgb = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_xgb)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.8347063728447839
Adjusted R^2: 0.8343565861896236
MAE: 0.8819311392954018
MSE: 3.300951381906551
RMSE: 1.8168520528393475

Test score
R^2: 0.6902555604880454
Adjusted R^2: 0.6887217503336533
MAE: 0.9587350743075962
MSE: 4.478613023860682
RMSE: 2.11627338117283


Feature importance 확인

In [11]:
f2 = pd.DataFrame({'data': X.columns, 'importance': best_xgb.feature_importances_}).sort_values(by='importance', ascending=False)
f2

,data,importance
1,일강수량(mm),0.223753
3,안개 계속시간(hr),0.115156
5,차량신호등,0.107114
4,보행신호등,0.106985
2,일 최심적설(cm),0.099186
10,교통안전표지,0.069433
9,정차금지대,0.039459
16,전체_추정교통량,0.038003
13,교통노드,0.037715
11,CCTV,0.032151


## 3-4 LightGBM Regressor

In [12]:
train = lgb.Dataset(X_train, label = y_train) 
test = lgb.Dataset(X_test, label = y_test)

base_score = [0.1,0.25,0.5]
booster=['gbtree','gblinear']
max_depth = [2, 3, 5, 10, 15]
min_child_weight=[1,2,3,4]
n_estimators = [100, 500, 900, 1100, 1500]
num_leaves = [32, 64, 128]

params = {'base_score':base_score,
          'booster':booster,
          'max_depth':max_depth,
          'min_child_weight':min_child_weight,
          'n_estimators':n_estimators,
          'num_leaves':num_leaves
         }

regressor = lgb.LGBMRegressor()
gd=RandomizedSearchCV(estimator=regressor,param_distributions=hyperparameter_grid,verbose=True)
gd.fit(X_train,y_train)
print(gd.best_score_)
print(gd.best_estimator_)
best_lgb = gd.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0.6872474235186672
LGBMRegressor(base_score=0.1, booster='gbtree', learning_rate=0.15,
              max_depth=10, min_child_weight=1, n_estimators=15)


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    7.5s finished


In [13]:
best_lgb.fit(X_train,y_train)
y_pred = best_lgb.predict(X_train)
print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = best_lgb.predict(X_test)

print('\nTest score')
acc_lgb = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_lgb)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.8048201241239352
Adjusted R^2: 0.8044070935848135
MAE: 0.976962810777752
MSE: 3.8977865758154477
RMSE: 1.9742812808248595

Test score
R^2: 0.6990526508761707
Adjusted R^2: 0.6975624026549248
MAE: 1.0318745838540808
MSE: 4.3514153778063545
RMSE: 2.086004644723102


Feature importance 확인

In [14]:
f3 = pd.DataFrame({'data': X.columns, 'importance': best_lgb.feature_importances_}).sort_values(by='importance', ascending=False)
f3

,data,importance
16,전체_추정교통량,80
1,일강수량(mm),64
3,안개 계속시간(hr),47
2,일 최심적설(cm),46
17,혼잡시간강도,38
14,건물면적,28
7,횡단보도,28
5,차량신호등,21
12,총인구,19
10,교통안전표지,18


## 3-5. SVM Regressor

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

svma = svm.SVR()
svma.fit(X_train, y_train)
y_pred = svma.predict(X_train)

print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = svma.predict(X_test)

acc_svm = metrics.r2_score(y_test, y_test_pred)
print('\nTest score')
print('R^2:', acc_svm)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

## 3-6. Voting

In [15]:
#Voting할 모델 선택
r1 = best_xgb 
r2 = RandomForestRegressor()
r3 = best_lgb
r4 = GradientBoostingRegressor()

vreg = VotingRegressor([('xgb', r1), ('rf', r2),('lgbm',r3),('gb',r4)],weights=[1,0.8,0.8,0.7])

vreg.fit(X_train,y_train)
y_pred = vreg.predict(X_train)
print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = vreg.predict(X_test)
print('\nTest score')
acc_vot = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_vot)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.872328212021789
Adjusted R^2: 0.8720580389458887
MAE: 0.7747869193696657
MSE: 2.5496346847141975
RMSE: 1.5967575535171887

Test score
R^2: 0.7162884540932116
Adjusted R^2: 0.714883555103852
MAE: 0.9484553995532743
MSE: 4.1022018878526145
RMSE: 2.0253893176010913


## 3-7. 결과 종합

In [16]:
models = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'XGBoost', 'LightGBM', 'Voting'],
    'R-squared Score': [acc_linreg*100, acc_rf*100, acc_xgb*100,acc_lgb*100, acc_vot*100]})
models.sort_values(by='R-squared Score', ascending=False)

,Model,R-squared Score
4,Voting,71.628845
1,Random Forest,70.995575
3,LightGBM,69.905265
2,XGBoost,69.025556
0,Linear Regression,35.983552


# 4. 결과 도출

Voting 결과가 가장 높은 LightGBM 을 최종 모델로 사용

In [17]:
dataset_encode['pred']=  best_lgb.predict(X.values)

In [18]:
#라벨인코더로 돌린 GID 복원
dataset_encode['gid'] = gid_lbe.inverse_transform(dataset_encode['gid'])

In [19]:
dataset_encode

,gid,acci_cnt,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry,pred
0,다바866110,0,0.0,0.00,0.0,0,0,0,0,0,0,0,0,13,0,146.59,4,8283.000,2.74000,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ...",0.292563
1,다바823157,0,0.0,0.00,0.0,3,4,2,3,0,0,14,0,267,1,5944.98,0,1181.625,38.47125,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ...",0.675835
2,다바928260,0,0.0,0.00,0.0,6,5,0,4,3,0,14,0,78,2,1487.49,40,3783.750,70.80250,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ...",1.924206
3,다바931203,2,2.1,1.65,0.0,1,3,1,1,2,0,5,0,0,0,1291.19,409,11596.000,98.50000,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ...",4.665356
4,다바836109,0,0.0,0.00,0.0,2,0,0,2,0,0,6,0,396,0,12875.52,0,678.875,65.44500,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ...",0.619875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,다바911108,1,0.0,0.00,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,286.500,94.86000,"MULTIPOLYGON (((127.40088 36.29447, 127.40088 ...",0.288078
12175,다바968288,0,0.0,0.00,0.0,0,0,0,0,0,0,2,0,0,0,0.00,0,2515.250,3.82500,"MULTIPOLYGON (((127.46429 36.45678, 127.46429 ...",0.270163
12176,다바906266,0,0.0,0.00,0.0,0,0,0,0,0,0,3,0,0,0,0.00,0,16135.250,5.13750,"MULTIPOLYGON (((127.39512 36.43691, 127.39512 ...",0.316242
12177,다바906193,0,0.0,0.00,0.0,0,0,0,0,0,0,2,1,0,0,0.00,0,14414.500,48.29500,"MULTIPOLYGON (((127.39521 36.37110, 127.39521 ...",0.353754


## 4-1. Pred를 지도로 시각화

In [20]:
ml_result = gpd.GeoDataFrame(dataset_encode, geometry = 'geometry')
ml_result_sort = ml_result.sort_values(by='pred', ascending = False)
ml_result_sort['x(경도)'] = ml_result_sort['geometry'].centroid.x
ml_result_sort['y(위도)'] = ml_result_sort['geometry'].centroid.y

# 위 / 경도 기준 50m 반경 데이터 구하기
ml_result_sort['geometry'] = ml_result_sort['geometry'].centroid.buffer(0.0005)
ml_result_sort

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  import sys
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Geometry is in a geographic C

,gid,acci_cnt,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry,pred,x(경도),y(위도)
3930,다바854185,49,2.387234,0.134043,0.125957,6,11,0,7,0,0,20,2,0,1,1035.50,5,17333.307692,71.327692,"POLYGON ((127.33831 36.36427, 127.33831 36.364...",37.197070,127.337813,36.364272
691,다바942159,42,3.819512,0.126829,0.096829,8,12,0,6,2,0,20,2,19,1,407.79,19,17067.200000,59.347000,"POLYGON ((127.43642 36.34093, 127.43642 36.340...",37.197070,127.435925,36.340925
1248,다바939179,51,2.891837,0.048980,0.194082,6,16,0,6,0,0,24,2,0,1,3637.26,0,14916.388889,57.920556,"POLYGON ((127.43307 36.35895, 127.43306 36.358...",35.723774,127.432566,36.358954
5436,다바877174,51,6.337255,0.109804,0.071961,5,12,0,5,4,0,6,0,43,1,437.06,24,16283.937500,95.613125,"POLYGON ((127.36397 36.35439, 127.36397 36.354...",35.105290,127.363468,36.354387
902,다바904156,50,1.842553,0.070213,0.008936,6,11,0,5,0,0,13,2,13,1,906.82,9,21535.833333,84.495833,"POLYGON ((127.39408 36.33819, 127.39408 36.338...",35.014709,127.393583,36.338190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877,다바826126,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,2,0,13,0,126.27,5,500.000000,13.050000,"POLYGON ((127.30723 36.31104, 127.30723 36.310...",0.246485,127.306734,36.311035
3822,다바911063,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,92.75,4,2806.250000,1.732500,"POLYGON ((127.40199 36.25435, 127.40199 36.254...",0.246485,127.401489,36.254353
10645,다바976304,0,0.000000,0.000000,0.000000,0,3,0,0,0,0,4,0,0,0,0.00,0,2245.000000,15.365000,"POLYGON ((127.47427 36.47166, 127.47427 36.471...",0.244338,127.473769,36.471662
8938,다바824036,0,0.000000,0.000000,0.000000,0,3,1,0,0,0,0,0,0,0,0.00,0,1517.500000,34.857500,"POLYGON ((127.30521 36.22989, 127.30521 36.229...",0.244338,127.304708,36.229892


In [21]:
ml_result_sort.to_file('map_result.geojson', driver= 'GeoJSON')

In [22]:
import folium

#대전시 중앙으로 오게 설정
center = (36.35, 127.4)
zoom = 12

#folium.Map으로 지도 그리기
m = folium.Map(location = center, 
               zoom_start = zoom,
               tiles = 'http://api.vworld.kr/req/wmts/1.0.0/BB730805-E115-3140-9139-BB79B7E3B74F/Base/{z}/{y}/{x}.png',
               attr = '대전시'
              )

#geojson에서 받은 값중 상위 100개만 지도에 표시

red = {'fillColor': '#ff0000', 'color': '#ff0000'}


folium.GeoJson(ml_result_sort.head(100),name='test2', style_function=lambda x:red).add_to(m)


# 시각화
m

In [23]:
result_map = gpd.read_file('map_result.geojson')

In [24]:
result_map = result_map.iloc[0:100, :]
result_map

,gid,acci_cnt,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,pred,x(경도),y(위도),geometry
0,다바854185,49,2.387234,0.134043,0.125957,6,11,0,7,0,0,20,2,0,1,1035.50,5,17333.307692,71.327692,37.197070,127.337813,36.364272,"POLYGON ((127.33831 36.36427, 127.33831 36.364..."
1,다바942159,42,3.819512,0.126829,0.096829,8,12,0,6,2,0,20,2,19,1,407.79,19,17067.200000,59.347000,37.197070,127.435925,36.340925,"POLYGON ((127.43642 36.34093, 127.43642 36.340..."
2,다바939179,51,2.891837,0.048980,0.194082,6,16,0,6,0,0,24,2,0,1,3637.26,0,14916.388889,57.920556,35.723774,127.432566,36.358954,"POLYGON ((127.43307 36.35895, 127.43306 36.358..."
3,다바877174,51,6.337255,0.109804,0.071961,5,12,0,5,4,0,6,0,43,1,437.06,24,16283.937500,95.613125,35.105290,127.363468,36.354387,"POLYGON ((127.36397 36.35439, 127.36397 36.354..."
4,다바904156,50,1.842553,0.070213,0.008936,6,11,0,5,0,0,13,2,13,1,906.82,9,21535.833333,84.495833,35.014709,127.393583,36.338190,"POLYGON ((127.39408 36.33819, 127.39408 36.338..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,다바873173,15,2.106667,0.346667,0.028000,2,4,1,1,4,0,6,0,0,0,509.29,4,11633.900000,94.926000,19.031984,127.359012,36.353481,"POLYGON ((127.35951 36.35348, 127.35951 36.353..."
96,다바925139,25,6.754167,0.191667,0.173750,2,14,1,3,0,0,8,2,9,1,161.44,18,10292.555556,65.599444,19.012992,127.417000,36.322883,"POLYGON ((127.41750 36.32288, 127.41750 36.322..."
97,다바907134,10,2.460000,0.030000,0.347000,5,6,0,3,4,0,12,0,48,1,476.92,26,15614.900000,88.666000,18.817945,127.396952,36.318359,"POLYGON ((127.39745 36.31836, 127.39745 36.318..."
98,다바960132,24,0.543478,0.730435,0.150870,3,5,1,3,0,0,9,2,30,1,314.75,15,15401.000000,84.676000,18.808274,127.455997,36.316593,"POLYGON ((127.45650 36.31659, 127.45649 36.316..."


In [25]:
len(result_map)

100

In [26]:
result_map_cordinate = result_map.loc[:, 'x(경도)':'y(위도)']
result_map_cordinate

,x(경도),y(위도)
0,127.337813,36.364272
1,127.435925,36.340925
2,127.432566,36.358954
3,127.363468,36.354387
4,127.393583,36.338190
...,...,...
95,127.359012,36.353481
96,127.417000,36.322883
97,127.396952,36.318359
98,127.455997,36.316593
